In [24]:
import cv2
import numpy as np
import os
import math
import tensorflow as tf
import idx2numpy as idx

# from matplotlib import pyplot as plt       # чтобы выводить промежуточные фото в jupyter
from PIL import Image
from tensorflow import keras
from keras.models import Sequential
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Reshape, LSTM, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras import backend as K
from keras.constraints import maxnorm

# Список всех настроечных параметров/констант
WORK_DIR = 'pass_photos'
TEMP_DIR = 'pass_temp'
DATESET_IMG = r"D:\work\test_comp_vision\datasets\!_lines_w25_dataset_images_100k.idx"
DATASET_CLS = r"D:\work\test_comp_vision\datasets\!_lines_w25_dataset_classes_100k.idx"
MODEL_PATH = 'ru_emnist_letters_100k_b64_e60.h5'
# TEST_FILE = 'pass_photos/1.jpeg'
IMG_HEIGHT = 1000            # требуемый размер фото для нормализации всех изображений
IMG_WIDTH = 600              # т.к. в задачу входит прочитать только ФИО, обрезаю серию/номер чтобы не усложнять распознавание
INDENT_LEFT = 220            # обрезаем фото т.к. без него получается лучше разделить фото на куски текста
INDENT_TOP = 40              # обрезаем лишнюю часть паспорта снизу
INDENT_BOTTOM = 120          # обрезаем нижние поля
SCALE_FACTOR = 8             # во сколько раз увеличиваем вырезанные слова для дальнейшей обработки букв
DATASET_SYMBOL_SIZE = 28     # размер изображений в тренировочном датасете      
LABELS = '0123456789АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя'
SYMBOLS_COUNT = len(LABELS)  # количество символов в датасете: 33 + 33 + 10 (заглавные, строчные, цифры)

In [8]:
# Функция для получения списка файлов из каталога с фотографиями (как в task_1 и task_2)
# TODO: переделать функцию, чтобы принимала в кач-ве параметра regex с перечислением искомых расширений файла
def get_files(directory: str) -> list:
    names = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg") or filename.endswith(".jpg") or filename.endswith(".png"):
            names.append(os.path.join(directory, filename))

    return names

In [9]:
# Масштабирование изображения
def scale_image(image, scale):     # принимаем объект изображения OpenCV
    
    # получаем текущий размер, вычисляем искомый и создаем измененное изображение
    height, width = image.shape[0], image.shape[1]
    img_width = int(width * scale)
    img_height = int(height * scale)
    img = cv2.resize(image, (img_width, img_height))
    #img = cv2.resize(image, (img_width, img_height), interpolation=cv2.INTER_CUBIC) # рекомендуют, но качество страдает
    
    return img

In [10]:
# TODO - это не пригодилось. Зря переусложнено. Но возможно без него я и получаю ошибку при распознавании
def normalize_img_size(image, size):
        h, w = image.shape[0], image.shape[1]     # сначала передается высота, потом ширина
        size_max = max(w, h)
        letter_square = 255 * np.ones(shape=[size_max, size_max], dtype=np.uint8)
        if w > h:
            y_pos = size_max//2 - h//2
            letter_square[y_pos:y_pos + h, 0:w] = letter_crop
        elif w < h:
            x_pos = size_max//2 - w//2
            letter_square[0:h, x_pos:x_pos + w] = letter_crop
        else:
            letter_square = letter_crop

        # Resize letter to 28x28 and add letter and its X-coordinate
        letters.append((x, w, cv2.resize(letter_square, (out_size, out_size), interpolation=cv2.INTER_AREA)))

In [11]:
# Нормализация размеров фотографии паспорта и вырезка нужной части для обработки
def cut_passport_info_area(image):     # принимаем объект изображения OpenCV
    
    # нормализуем фото к нужному размеру
    old_height = image.shape[0]     # получаем исходную высоту
    resize_scale = IMG_HEIGHT / old_height       # считаем коэффициент масштабирования изображения до требуемого
    img = scale_image(image=image, scale=resize_scale)
    new_width = img.shape[1]      # получаем новую ширину
    
    # обрезаем паспорт до страницы с фото
    x0 = INDENT_LEFT                            # отступ слева, т.к. корочка и фото нам не важны
    y0 = IMG_HEIGHT // 2 + INDENT_TOP           # обрезка сверху, т.к. верхняя страница с местом выдачи нам не важна 
    x1 = new_width if new_width < IMG_WIDTH else IMG_WIDTH   # обрезаем все лишнее справа, если есть разворот с пропиской
    y1 = IMG_HEIGHT - INDENT_BOTTOM
    img = img[y0:y1, x0:x1]              # сохраняем вырезанный кусок изображения для передачи
    
    return img

In [12]:
# Подготовка изображений для распознавания текста
def normalize_color(image):         # принимаем объект изображения OpenCV
    
    # обесцвечиваем, если картинка цветная
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
    else:
        gray = image
    
    # Размытие для снижения количества шумов. Эксперимент показал, что без него буквы детектируются лучше
    # blur = cv2.GaussianBlur(gray, (5,5), 0)         # коэффициент размытия подобран вручную
    
    # Очередность преобраозвания найдена опытным путем
    kernel = np.ones((5,5), 'uint8')
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))  # не знаю в чем разница, но так работает хуже
    
    # В теории erode - делает буквы тоньше, а dilate - толще: https://docs.opencv.org/3.4/db/df6/tutorial_erosion_dilatation.html
    img_block = cv2.erode(gray, kernel, iterations=1)   # Но на практике "жирность" букв при этой операции повышается
    #img_block = cv2.dilate(img_block, kernel, iterations=1)  # А тут - наоборот
    
    # TODO - поиграться с настройками, чтобы выдавать на выход именно контраст. Сейчас это только для детекции границ букв
    _, img_block = cv2.threshold(img_block, 0, 255, cv2.THRESH_OTSU, cv2.THRESH_BINARY_INV) # Повышаем контраст
    img_block = cv2.morphologyEx(img_block, cv2.MORPH_OPEN, kernel, iterations=1) # Снижаем шум на фоне
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    
    """
    # Попытка найти лучший вариант детекции и выходного изображения. Оставил для дальнейших тестов
    # Grayscale, Gaussian blur, Otsu's threshold
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Morph open to remove noise and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    erosion = cv2.erode(gray, kernel, iterations = 1)
    dilation = cv2.dilate(gray, kernel, iterations = 1)
    invert = 255 - closing
    
    # Повышение контраста
    if len(image.shape) > 2:
        imghsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        imghsv[:,:,2] = [[max(pixel - 25, 0) if pixel < 190 else min(pixel + 25, 255) for pixel in row] for row in imghsv[:,:,2]]
        contrast = cv2.cvtColor(imghsv, cv2.COLOR_HSV2BGR)
        gray_contrast = cv2.cvtColor(contrast, cv2.COLOR_BGR2GRAY)    # преобразуем в ЧБ
        
    # при коэффициенте 3 - лучше распознается Васлевский, при 5 - Соколов и Юмакаева
    img_symbol = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 3, 2)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_TOZERO+cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    """
    

    return img_block, gray      # Возвращаем контрастную картинку с разбивкой на блоки и простое ЧБ изображение

In [74]:
# Выделяем элементы текста из изображения
def search_blocks(image, limit: int, sort_by: str, sort_reverse=False):
    #::limit:: - необходим чтобы указать на сколько мелкие символы нам не нужно распознавать
    
    height, width = image.shape[0], image.shape[1]
    # получаем контуры больших пятен на изображении, внутри которых спрятан текст
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # contours, hierarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # другая вариация
    
    # print(f'Count of Block counoturs: {len(contours)}')
    blocks = []
    for idx, contour in enumerate(contours):
        (x, y, w, h) = cv2.boundingRect(contour)
        # print("R", x, y, w, h, hierarchy[0][idx])
        # hierarchy[i][0]: следующий контур текущего уровня
        # hierarchy[i][1]: предыдущий контур текущего уровня
        # hierarchy[i][2]: первый вложенный элемент
        # hierarchy[i][3]: родительский элемент
        # if hierarchy[0][idx][3] == 0:               # если элемент не является самым крупным
        # cv2.rectangle(image, (x, y), (x + w, y + h), (70, 0, 0), 1) # для контрольной картинки
        
        if limit < h < height and limit < w < width:    # игнорируем маленькие блоки, а также блок размером с изображение
            block = image[y:y + h, x:x + w]     # вырезаем найденный блок из изображения
            
            # сохраняем габариты и изображение блока в список блоков. Загоняем в словарь, чтобы проще сортировать
            # todo: По 'x' мы определяем очередность букв, ведь чем "левее буква", тем меньше ее 'x'. Также можно по 'y'
            blocks.append({'idx': idx, 'y': y, 'h': h, 'x': x, 'w': w, 'block': block})
    
    # Сортируем по нужному ключу: 'y' для вертикали или 'x' по горизонтали. Так же можно и по индексу или размерам
    blocks.sort(key=lambda x: x.get(sort_by), reverse=sort_reverse)
    # print(blocks)
    return blocks    

In [80]:
"""
Если детектор букв выдает нам слишком "широкий" блок - значит он склеил несколько соседних букв.
Это возможно по двум причинам:
1. Плохое качество печати/изображения, тогда действительно соседние буквы сливаются даже для человеческого взгляда.
2. Плохое качество фильтра определения границ букв. С этим еще нужно поработать - поковырять параметры в normalize_color()
Выход - решать проблему математически (на вскидку). Если ширина больше высоты на определенную константу (подобрана руками)
то делим изображение на расчетное количество элементов.
Это не панацея, т.к. в зависимости от шрифта буква "Ж" может быть шире, чем сочетание "СТ". С английскими "ij" еще хуже.
"""
def cut_blocks(image):
    height, width = image.shape[0], image.shape[1]
    C = 1.2       # просто коэффициент, рассчитанный на широкие буквы вроде Ж, М, Ш и т.д., чтобы их не резало
    if width < height*C:
        # print(f'One symbol is True')
        return [image]
    else:
        #print(f'One symbol is FALSE')
        result = []
        y, h, = 0, height      # высота и верхняя точке среза - всегда неизменны
        symbol_count = math.ceil(width / height)    # округляем символы до большего целого
        symbol_width = math.floor(width / symbol_count)   # округляем ширину в пикселях до меньшего целого
        
        for i in range(symbol_count):
            x = i * symbol_width
            result.append(image[y:h, x:x+symbol_width])
            # print(f'y = {y}, h = {h}, x = {x}, symbol_width = {x+symbol_width}, width = {width}')
            # print(f'symbol {i} is:\n{result[i]}')
            
        # print(f'Count of separeted symbols: {len(result)}')
        return result

### Детекция данных из паспорта и сохранение фоток букв в файлы

In [30]:
passports = get_files(WORK_DIR)
print(passports[:5])

['pass_photos\\0.jpeg', 'pass_photos\\1.jpeg', 'pass_photos\\2.jpeg', 'pass_photos\\3.jpeg', 'pass_photos\\4.png', 'pass_photos\\5.jpeg', 'pass_photos\\6.jpeg', 'pass_photos\\7.jpeg', 'pass_photos\\8.jpeg', 'pass_photos\\9-1.jpeg']


In [81]:
def passport_data_parser(work_dir: str, count: int):
    # Запускаем цикл по всем фото в рабочей папке
    try:     # TODO добавить проверку на существование файла
        passports = get_files(work_dir)
    except Exception as e:
        return e
        
    export_words = []
    count = min(count, len(passports))     # Что меньше - по такой индекс и забираем фотки (для тестов)
    
    for id_p, passport in enumerate(passports[:count]):     # идем по списку путей к изображениям (ограничив длину списка)
        temp_dir = os.path.join(TEMP_DIR, str(id_p))
        if not os.path.exists(temp_dir):
            os.mkdir(temp_dir)                              # создаем папку для сохранения промежуточных картинок

        print(f'==== Image {id_p}.jpg =====')
        image = cut_passport_info_area(cv2.imread(passport))     # получаем кусок паспорта с ФИО
        img_blocks, img_gray = normalize_color(image=image)      # img_gray используем для передачи дальше
        
        # TODO - убрать сохранение промежутоных файлов, они используются только для визуального контроля
        cv2.imwrite(f'{TEMP_DIR}/{id_p}_blocs.jpg', img_blocks)
        cv2.imwrite(f'{TEMP_DIR}/{id_p}_symbols.jpg', image)

        words = search_blocks(image=img_blocks, limit=15, sort_by='y')   # ищем блоки на картинке с жирными буквами
        # cv2.imshow('The First Word', words[0]['block'])
        # cv2.waitKey(0)
        # print(f'Count of words: {words}')

        # получаем все обнаруженные слова из файла, в котором читаются символы
        for id_w, word in enumerate(words[:3]):    # можно забирать только первые 3 слова ФИО
            export_words.append([None])      # добавляем вложенный список для каждого слова
            # из словаря обнаруженного блока текста забираем координаты и размер блока
            y, h, x, w = word['y'], word['h'], word['x'], word['w']
            img_word = img_gray[y:y + h, x:x + w]     # вырезаем слово из серой картинки по его координатам
            # img_word = image[y:y + h, x:x + w]     # вариант с повышением контраста, поэкспериментировать
            
            img_word = scale_image(img_word, SCALE_FACTOR)   # увеличиваем изображение, чтобы детектировать буквы
            cv2.imwrite(os.path.join(temp_dir, f'{id_w}.jpg'), img_word)   #сохраняем файлы только для контроля

            word_blocks, word_text = normalize_color(image=img_word) # прогоняем через детектор увеличенное фото слова
            #word_blocks, word_text = normalize_color(image=word_text)    # вариант с повышением контраста
            symbols = search_blocks(image=word_blocks, limit=SCALE_FACTOR*10, sort_by='x')
            # print(f'Count of symbols: {len(symbols)}')
            
            # TODO - частичный повтор кода. Придумать как переделать, чтобы не дублировать функционал
            for id_s, symbol in enumerate(symbols):
                # TODO - убрать сохранение промежутоных файлов, они используются только для визуального контроля
                word_dir = os.path.join(temp_dir, str(id_w))     # создаем очередную вложенную папку для котроля
                if not os.path.exists(word_dir):
                    os.mkdir(word_dir)

                y, h, x, w = symbol['y'], symbol['h'], symbol['x'], symbol['w']
                img_symbol = word_text[y:y + h, x:x + w]
                #cv2.imwrite(os.path.join(word_dir, f'{symbol[0]}-{e}.jpg'), img_symbol)

                # Доп. проверка на случай, если буквы плохо отделились
                for id_o, one_symbol in enumerate(cut_blocks(img_symbol)):
                    # one_symbol = cv2.resize(one_symbol, (DATASET_SYMBOL_SIZE, DATASET_SYMBOL_SIZE), interpolation=cv2.INTER_AREA)
                    one_symbol = cv2.resize(one_symbol, (DATASET_SYMBOL_SIZE, DATASET_SYMBOL_SIZE))
                    cv2.imwrite(os.path.join(word_dir, f'{id_s}-{id_o}.jpg'), one_symbol)
                    export_words[id_w].append(one_symbol)
                    
    return export_words

In [93]:
symbols = []
symbols.append(passport_data_parser(work_dir=WORK_DIR, count=1))   # Для теста разбираем только 1 паспорт
print(len(symbols[0]))

==== Image 0.jpg =====
3


### Готовим train/test

In [16]:
# Подгатавливаем модель для распознавания букв из датасетов по аналогии с EMNIST
def main_model(img_size, lb_count):
    model = Sequential()
    model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='valid',
                            input_shape=(img_size, img_size, 1), activation='relu'))
    model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(lb_count, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
    return model

In [17]:
def labels_to_int(labels=LABELS) -> dict:
    label_nums = {}
    for i, lab in enumerate(labels):
        label_nums[lab] = i
    print(label_nums)
    return label_nums

In [18]:
labels_comparison = labels_to_int()     # получаем сопоставление символа к коду из датасета
labels_symbol = list(labels_comparison.keys())     # и получаем отдельно список по символам и кодам
labels_class = list(labels_comparison.values())

# Загружаем датасеты картинок и их классов
ds_images = idx.convert_from_file(DATESET_IMG)
ds_classes = idx.convert_from_file(DATASET_CLS)

# Разбиваем выборки на train, test
# TODO переписать под train, test, validate
X_train, X_test = np.split(ds_images, [int(.85*len(ds_images))])
y_train, y_test = np.split(ds_classes, [int(.85*len(ds_classes))])

#print(labels_symbol)
#print(labels_class)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'А': 10, 'Б': 11, 'В': 12, 'Г': 13, 'Д': 14, 'Е': 15, 'Ё': 16, 'Ж': 17, 'З': 18, 'И': 19, 'Й': 20, 'К': 21, 'Л': 22, 'М': 23, 'Н': 24, 'О': 25, 'П': 26, 'Р': 27, 'С': 28, 'Т': 29, 'У': 30, 'Ф': 31, 'Х': 32, 'Ц': 33, 'Ч': 34, 'Ш': 35, 'Щ': 36, 'Ъ': 37, 'Ы': 38, 'Ь': 39, 'Э': 40, 'Ю': 41, 'Я': 42, 'а': 43, 'б': 44, 'в': 45, 'г': 46, 'д': 47, 'е': 48, 'ё': 49, 'ж': 50, 'з': 51, 'и': 52, 'й': 53, 'к': 54, 'л': 55, 'м': 56, 'н': 57, 'о': 58, 'п': 59, 'р': 60, 'с': 61, 'т': 62, 'у': 63, 'ф': 64, 'х': 65, 'ц': 66, 'ч': 67, 'ш': 68, 'щ': 69, 'ъ': 70, 'ы': 71, 'ь': 72, 'э': 73, 'ю': 74, 'я': 75}
X_train: (85000, 28, 28)
X_test: (15000, 28, 28)
y_train: (85000,)
y_test: (15000,)


In [19]:
X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))

#k = 10
#X_train = X_train[:X_train.shape[0] // k]
#y_train = y_train[:y_train.shape[0] // k]
#X_test = X_test[:X_test.shape[0] // k]
#y_test = y_test[:y_test.shape[0] // k]

# Нормализация - ХЗ что такое, разобраться
X_train = X_train.astype(np.float32)
X_train /= 255.0
X_test = X_test.astype(np.float32)
X_test /= 255.0

# "маска" на которую будут созданы предсказание категорий
x_train_cat = keras.utils.to_categorical(y_train, len(labels_comparison))
y_test_cat = keras.utils.to_categorical(y_test, len(labels_comparison))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, len(labels_comparison))
print(x_train_cat.shape, y_test_cat.shape)

(85000, 28, 28, 1) (85000,) (15000, 28, 28, 1) (15000,) 76
(85000, 76) (15000, 76)


## Обучаем модель на датасете

In [12]:
# Set a learning rate reduction
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

# Загружаем в модель размер изображений и количество классов
model = main_model(DATASET_SYMBOL_SIZE, len(labels_comparison))

model.fit(X_train, x_train_cat, validation_data=(X_test, y_test_cat), callbacks=[learning_rate_reduction], batch_size=64, epochs=30)

model.save('ru_emnist_letters_100k_b64_e30.h5')

Epoch 1/60
1329/1329 [==============================] - 103s 77ms/step - loss: 4.3291 - accuracy: 0.0459 - val_loss: 4.3275 - val_accuracy: 0.0503 - lr: 0.0010
Epoch 2/60
1329/1329 [==============================] - 107s 81ms/step - loss: 4.3258 - accuracy: 0.0460 - val_loss: 4.3241 - val_accuracy: 0.0503 - lr: 0.0010
Epoch 3/60
1329/1329 [==============================] - 106s 80ms/step - loss: 4.3221 - accuracy: 0.0468 - val_loss: 4.3201 - val_accuracy: 0.0503 - lr: 0.0010
Epoch 4/60
1328/1329 [============================>.] - ETA: 0s - loss: 4.3178 - accuracy: 0.0461
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1329/1329 [==============================] - 106s 80ms/step - loss: 4.3178 - accuracy: 0.0460 - val_loss: 4.3154 - val_accuracy: 0.0503 - lr: 0.0010
Epoch 5/60
1329/1329 [==============================] - 107s 80ms/step - loss: 4.3140 - accuracy: 0.0467 - val_loss: 4.3126 - val_accuracy: 0.0503 - lr: 5.0000e-04
Epoch 6/60
1329/1329 [===========

Epoch 44/60
1329/1329 [==============================] - 122s 92ms/step - loss: 4.2819 - accuracy: 0.0464 - val_loss: 4.2820 - val_accuracy: 0.0503 - lr: 1.0000e-05
Epoch 45/60
1329/1329 [==============================] - 122s 92ms/step - loss: 4.2819 - accuracy: 0.0456 - val_loss: 4.2818 - val_accuracy: 0.0503 - lr: 1.0000e-05
Epoch 46/60
1329/1329 [==============================] - 119s 89ms/step - loss: 4.2816 - accuracy: 0.0465 - val_loss: 4.2816 - val_accuracy: 0.0503 - lr: 1.0000e-05
Epoch 47/60
1329/1329 [==============================] - 121s 91ms/step - loss: 4.2813 - accuracy: 0.0455 - val_loss: 4.2813 - val_accuracy: 0.0503 - lr: 1.0000e-05
Epoch 48/60
1329/1329 [==============================] - 124s 93ms/step - loss: 4.2810 - accuracy: 0.0454 - val_loss: 4.2811 - val_accuracy: 0.0503 - lr: 1.0000e-05
Epoch 49/60
1329/1329 [==============================] - 123s 92ms/step - loss: 4.2809 - accuracy: 0.0459 - val_loss: 4.2809 - val_accuracy: 0.0503 - lr: 1.0000e-05
Epoch 50/6

In [94]:
# model_saved = keras.models.load_model('ru_emnist_letters_100k_b64_e60.h5')

def predict_img(model, img):
    
    # TODO - этот код из примера. Возможно нужно раскомментировать последний этап, и без него не работает мой предикт
    # img_arr = np.expand_dims(img, axis=0)
    # img_arr = 1 - img_arr/255.0
    # img_arr[0] = np.rot90(img_arr[0], 3)
    # img_arr[0] = np.fliplr(img_arr[0])
    # img_arr = img_arr.reshape((1, 28, 28, 1))
    img = img.reshape((1, 28, 28, 1))

    predict = model.predict(img)
    result = np.argmax(predict, axis=1)     # получаем индекс класса с наибольшей предсказанной вероятностью
    
    #TODO изменить на использование словаря соответствия класса и буквы. сейчас тупо по индексу класса забираю букву
    # return chr(emnist_labels[result[0]])
    print(labels_symbol[result[0]])
    return labels_symbol[result[0]]

In [36]:
# не нужно, убрать
def img_to_str(model, word_by_imgs):
    s_out = ""
    
    for letter in word_by_imgs:
        s_out += predict_img(model, letter)
    return s_out

In [95]:
word_images_paths = get_files(r'D:\work\test_comp_vision\test_for_MindSet\pass_temp\0\0_a')
word_images = []     # сюда собираем список всех картинок для одного слова
predicted_word = ''

# Если только что занимались созданием модели - будет запущена она. Если нет - будет запущена версия с диска  
if 'model' in locals() or 'model' in globals():
    print("The model just created will be used.")
else:
    try:
        print("The Model has not been created in the current session. Loading the saved model.")
        model = keras.models.load_model(MODEL_PATH)
    except NameError as ne:
        print(ne)
    except Exception as e:     # TODO Добавить обработку отсутствия файла
        print(e)

for letter_path in word_images_paths:
    #word_images.append(cv2.imread(letter_path))
    with Image.open(letter_path) as image:       # открываем картинку по ссылке, преобразуем в массив
        img_to_arr = np.asarray(image)
        img_to_arr = np.asarray([img_to_arr])    # преобразуем загруженную картинку к необходимой Модели форме 
        print(img_to_arr.shape)
        #word_images.append(np.asarray(image))
        
        predicted_word += predict_img(model=model, img=img_to_arr)
        
        
#print(word_images[0])

    
#predicted_word = img_to_str(model=model_saved, word_by_imgs=word_images)

print(predicted_word)

The model just created will be used.
(1, 28, 28)
1/1 [==============================] - 0s 130ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 21ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 18ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 33ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 17ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 21ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 20ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 20ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 17ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 18ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 22ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 19ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 17ms/step
О
(1, 28, 28)
1/1 [==============================] - 0s 18ms/step
О
(1, 28, 28)
1/1 [=====================